In [5]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

 #Session = sagemaker.Session(
    #boto3.session.Session(region_name="us-east-1")  # Change to your region
#)


boto_session = boto3.Session(region_name="ap-south-1")  # 👈 Set region explicitly
sagemaker_session = sagemaker.Session(boto_session)

sm_boto3 = boto3.client("sagemaker" ,region_name="ap-south-1")
#sess = sagemaker.Session()
region = sagemaker_session.boto_session.region_name
bucket = 'sagemaker-ap-south-1-927311782619'
print("using bucket " + bucket)

using bucket sagemaker-ap-south-1-927311782619


In [6]:
df = pd.read_csv("train.csv")

In [7]:
df.head(20)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0,0.5,1,3,0,22,0.7,164,1,...,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0,1.7,0,4,1,10,0.8,139,8,...,381,1018,3220,13,8,18,1,0,1,3
7,1954,0,0.5,1,0,0,24,0.8,187,4,...,512,1149,700,16,3,5,1,1,1,0
8,1445,1,0.5,0,0,0,53,0.7,174,7,...,386,836,1099,17,1,20,1,0,0,0
9,509,1,0.6,1,2,1,9,0.1,93,5,...,1137,1224,513,19,10,12,1,0,0,0


In [8]:
df.shape

(2000, 21)

In [9]:
df.isnull().mean()* 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [10]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [11]:
label  = features.pop(-1)
label

'price_range'

In [12]:
x = df[features]
y = df[label]

In [13]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [14]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [15]:
x.shape

(2000, 20)

In [16]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [19]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train


testX = pd.DataFrame(X_test)
testX[label] = y_test

In [20]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [21]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [22]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [23]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [28]:
trainX.to_csv("train-V-1.csv",index=False)
testX.to_csv("test-V-1.csv",index=False)


In [25]:
bucket

'sagemaker-ap-south-1-927311782619'

In [26]:
bucket

'sagemaker-ap-south-1-927311782619'

In [29]:
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sagemaker_session.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sagemaker_session.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

print(trainpath)
print(testpath)

s3://sagemaker-ap-south-1-927311782619/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://sagemaker-ap-south-1-927311782619/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


In [ ]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score,
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    #hyperparameters sent by the client are passed as command-line arguments

    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    #Data, model, and output directories

    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args,_ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)


    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))


    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]


    print('Column order: ')
    print(features)
    print()

    print("Label column is: ",label)
    print()


    print("Data Shape: ")
    print()
    print("----SHAPE OF TRAINING DATA (85%)----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("----SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()


    print("Training RandomForest Model....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)









Overwriting script.py


In [39]:
import boto3
from sagemaker.sklearn.estimator import SKLearn

session = sagemaker.Session(
    boto3.Session(region_name="ap-south-1")
)

# Then pass this to your estimator if needed:
sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::927311782619:role/ExecutionRole",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="0.23-1",
    base_job_name="RF-custom-sklearn",
    hyperparameters={"n_estimators": 100, "random_state": 0},
    use_spot_instances=False,
    #max_wait=7200,
    max_run=3600,
    sagemaker_session=session  # <-- This line is optional if region is set globally
)


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [40]:
bucket

'sagemaker-ap-south-1-927311782619'

In [41]:
sklearn_estimator.fit({
    "train": "s3://sagemaker-ap-south-1-927311782619/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv"
})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2025-06-07-08-43-35-588


2025-06-07 08:43:43 Starting - Starting the training job...
2025-06-07 08:44:17 Downloading - Downloading input data...
2025-06-07 08:44:43 Downloading - Downloading the training image...
2025-06-07 08:45:28 Training - Training image download completed. Training in progress..2025-06-07 08:45:32,855 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-07 08:45:32,858 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 08:45:32,902 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-07 08:45:33,059 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 08:45:33,071 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 08:45:33,083 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 08:45:33,092 sagemaker-training-toolkit INFO     Invoking user script
Trai

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sklearn_estimator.fit({                                                                      │
│   2 │   "train": "s3://sagemaker-ap-south-1-927311782619/sagemaker/mobile_price_classificati     │
│   3 })                                                                                           │
│   4                                                                                              │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py:167 in wrapper        │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py:138 in wrapper        │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\workflow\pipeline_context.py:346 in wrapper          │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                      

In [42]:
sklearn_estimator.fit({"train":trainpath, "test":testpath}, wait=True)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2025-06-07-09-13-00-792


2025-06-07 09:13:04 Starting - Starting the training job...
2025-06-07 09:13:37 Downloading - Downloading input data...
2025-06-07 09:14:02 Downloading - Downloading the training image...
2025-06-07 09:14:48 Training - Training image download completed. Training in progress..2025-06-07 09:14:52,217 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-07 09:14:52,220 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 09:14:52,259 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-07 09:14:52,403 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 09:14:52,416 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 09:14:52,427 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-07 09:14:52,436 sagemaker-training-toolkit INFO     Invoking user script
Trai

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sklearn_estimator.fit({"train":trainpath, "test":testpath}, wait=True)                       │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py:167 in wrapper        │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\telemetry\telemetry_logging.py:138 in wrapper        │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using SKlearn Custom Script in             │
│ Sagemaker\myenv\lib\site-packages\sagemaker\workflow\pipeline_context.py:346 in wrapper          │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ c:\Users\Veneel\Downloads\Mobile Price Classification using